In [1]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(10.0,8.0)
import seaborn as sns
from scipy import stats
from scipy.stats import norm

In [49]:
#load data

train1 = pd.read_excel("SLR3.xlsx",header=0)


In [50]:
from sklearn.model_selection import train_test_split
train,  test=train_test_split(train1,test_size=0.33)

In [51]:
test.shape

(619, 4)

In [52]:
train.head()

,ID,Title,Abstract,Result
339,343,Medical Treatment in Elderly Patients with Non...,PURPOSE/OBJECTIVES: To examine how routine lab...,Exclude
394,400,High BMI and low HDL-C predict the chemotherap...,RATIONALE: Myeloid-derived suppressor cells (M...,Exclude
863,879,Prospective cohort study of clinical character...,NaN,Include
1547,1871,How and when to use genetic markers for nonsma...,NaN,Exclude
1278,1600,"Structural, biochemical, and clinical characte...",NaN,Exclude


In [53]:
train.describe(include=[np.object])

,Title,Abstract,Result
count,1255,249,1255
unique,1253,249,2
top,High TBX2 expression predicts poor prognosis i...,PURPOSE: BRAF mutations are found in a subset ...,Exclude
freq,2,1,846


In [54]:
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

from sklearn.feature_extraction import text 

stop = text.ENGLISH_STOP_WORDS.union(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','%'])

In [55]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True)

In [56]:
train['Abstract'].fillna("Empty",inplace=True)
test['Abstract'].fillna("Empty",inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [57]:
%%time
vec = vectorizer.fit_transform(train['Abstract'])
testvec=vectorizer.transform(test['Abstract'])

Wall time: 4.03 s


In [58]:
vectorizerTitle = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True)

In [59]:
vecTitle =  vectorizerTitle.fit_transform(train['Title'])
testvecTitle=vectorizerTitle.transform(test['Title'])

In [60]:
import scipy.sparse as sps

In [61]:
new=sps.hstack((vec,vecTitle))
testnew=sps.hstack((testvec,testvecTitle))

In [15]:
vocab=pd.DataFrame(columns=vectorizer.get_feature_names(),data=vec.A)
vocab.to_csv("Vocab.csv")

In [28]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(objective='multi:softmax', n_estimators=300, learning_rate=0.05)

C:\Users\manshubh.singh\AppData\Local\conda\conda\envs\my_root\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#model.set_params(num_class=3)
#xgb_param['num_class'] = 3
model.get_params().keys()

In [29]:
%%time
xgbfit = model.fit(new,train['Result'])

XGBoostError: b'value 0for Parameter num_class should be greater equal to 1'

In [62]:
%%time
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(new,train['Result'])
print(logistic)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Wall time: 23 ms


In [63]:
test['PredicResult']=logistic.predict(testnew)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
logistic.score(testnew,test['Result'])

0.68982229402261708

In [ ]:
xgbfit.score(testnew,test['Class'])

In [ ]:
xgbresult=pd.DataFrame()
xgbresult['ID']=train['ID']
xgbresult=pd.DataFrame(index=train.ID,data=xgbfit.score(new,train['Class']))
#xgbresult[c]=xgbresult[c].applymap(stand)
xgbresult.head(10)

In [27]:
result=pd.DataFrame()
result['ID']=test['Cit_ID']
result=pd.DataFrame(index=test.index,data=logistic.predict(testnew),columns=['Result'])
#result[c]=result[c].applymap(stand)
result.head(10)

,Result
38,Exclude
349,Exclude
393,Exclude
8,Exclude
77,Exclude
343,Exclude
103,Exclude
439,Exclude
51,Exclude
529,Exclude


In [ ]:
#test.to_excel("SLR3Result.xlsx")